In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

path = '../raw'
path_lojas_atuais = f'{path}/lojas_atuais.csv'
path_faturamento_lojas_atuais = f'{path}/faturamento_lojas_atuais.csv'
path_cenarios_expansao = f'{path}/cenarios_expansao.csv'

lojas = pd.read_csv(path_lojas_atuais)
faturamento = pd.read_csv(path_faturamento_lojas_atuais)
expansao = pd.read_csv(path_cenarios_expansao)


In [ ]:
# Arrumando o dataset para ficar do jeito que vamos treinar 

In [ ]:
path_ibge = 'tabelas_ibge_uteis/tabela392_csv.csv'

ibge = pd.read_csv(path_ibge, sep=';', encoding='latin1')

In [ ]:
#ibge_values = ibge['ï»¿Areas'].values
#ibge.drop(['ï»¿Areas'], axis=1, inplace=True)
#ibge['Areas'] = pd.Series(ibge_values)
ibge.head()


In [ ]:
ibge.columns
ibge = ibge.rename(columns={'Column5': 'renda_media', 'Column11': 'cod_ap'})
ibge.columns

In [ ]:
ibge = ibge[['cod_ap', 'renda_media']]
ibge.head()
ibge.dtypes

In [ ]:
lojas.sample(30)

# IMPORTANTE -> FEATURES 13 a 15 ou todas estao presentes ou nenhuma esta presente -> tranformar em feature binaria posteriormente

In [ ]:
lojas.isnull().sum()
print(lojas.dtypes)

In [ ]:
# mudar os types:

# observacao feat 12 e binaria na vdd mas vamos deixar assim, pq
# ainda teremos que lidar com missing values, entao ter 3 categorias
# na vdd
lojas['cod_loja'] = lojas['cod_loja'].astype(str)
lojas['cod_ap'] = lojas['cod_ap'].astype(str)
lojas['cod_municipio'] = lojas['cod_municipio'].astype(str)

In [ ]:
print(lojas.shape)
print(lojas.info())
print(lojas.duplicated().any())

In [ ]:
#  lojas onde foi inputado -1 (nao se conhece dados geograficos)
lojas[lojas['cod_ap'] == '-1']

In [ ]:
faturamento.head(15)

In [ ]:
print(faturamento.shape)
print(faturamento.info())
print(faturamento.duplicated().any())

In [ ]:
expansao.head()

In [ ]:
print(expansao.shape)
print(expansao.info())
print(expansao.duplicated().any())

In [ ]:
faturamento['cod_loja'].nunique() 

In [ ]:
len(lojas)

In [ ]:
len(expansao)

In [ ]:
faturamento.isnull().values.any()

In [ ]:
faturamento.describe()

In [ ]:
faturamento.head()

In [ ]:
# colocar as categorias para cada mes

cat_concat_list = []
loja_ano_mes_cont_list = []
i = 0
for ((cod_loja, ano, mes), gp) in faturamento.groupby(['cod_loja', 'ano', 'mes']):
    month_categories = pd.Series(gp['qtde'].values, index=gp['categoria'].values).to_dict()
    cat_df = pd.DataFrame(month_categories, index=[i])
    cat_concat_list.append(cat_df)
    loja_ano_mes_cont_list.append({'cod_loja':cod_loja, 'ano':ano, 'mes':mes})
    i += 1 
cat_full = pd.concat([*cat_concat_list])
loja_ano_mes_cont_full = pd.DataFrame(loja_ano_mes_cont_list)
loja_ano_mes_cont_full.head(10)


In [ ]:
loja_ano_mes_cont_full.shape

In [ ]:
cat_full.shape

In [ ]:
faturamento = faturamento.groupby(['cod_loja', 'ano', 'mes'])[['qtde', 'receita']].sum().reset_index(drop = True)
faturamento.shape

In [ ]:
faturamento = pd.concat([loja_ano_mes_cont_full, faturamento, cat_full], axis=1)
faturamento.shape

In [ ]:
faturamento.head()

In [ ]:
faturamento.to_csv('../final/estrabico.csv')

In [ ]:
# todas as combinacoes possiveis de loja, ano, mes
# garantir que nao vai faltar nada

from itertools import product

loja_unique = faturamento['cod_loja'].unique()
ano_unique = faturamento['ano'].unique()
mes_unique = faturamento['mes'].unique()

all_combinations = np.vstack(list(product(loja_unique, ano_unique, mes_unique)))

combinations_faturamento = pd.DataFrame(all_combinations, columns=['cod_loja', 'ano', 'mes'])

In [ ]:
combinations_faturamento = combinations_faturamento.sort_values(by=['cod_loja', 'ano', 'mes'])

In [ ]:
faturamento = pd.merge(faturamento, combinations_faturamento, on=['cod_loja', 'ano', 'mes'], how='right')

In [ ]:
faturamento.head()

In [ ]:
faturamento.loc[faturamento['ano'] == 2016, 'mes_cont'] = faturamento.loc[faturamento['ano'] == 2016, 'mes']
faturamento.loc[faturamento['ano'] == 2017, 'mes_cont'] = 12 + faturamento.loc[faturamento['ano'] == 2017, 'mes']

In [ ]:
print(faturamento['CATEG_08'].max())
print(faturamento['CATEG_08'].mean())
print(faturamento['CATEG_07'].max())
print(faturamento['CATEG_07'].mean())
print(faturamento['CATEG_06'].max())
print(faturamento['CATEG_06'].mean())
print(faturamento['CATEG_05'].max())
print(faturamento['CATEG_05'].mean())
print(faturamento['CATEG_04'].max())
print(faturamento['CATEG_04'].mean())
print(faturamento['CATEG_03'].max())
print(faturamento['CATEG_03'].mean())
print(faturamento['CATEG_02'].max())
print(faturamento['CATEG_02'].mean())
print(faturamento['CATEG_01'].max())
print(faturamento['CATEG_01'].mean())

In [ ]:
faturamento['cod_loja'] = faturamento['cod_loja'].astype(str)

In [ ]:
faturamento = pd.merge(faturamento, lojas, on='cod_loja', how='inner')
faturamento.head(12)

In [ ]:
faturamento.shape

In [ ]:
faturamento.isnull().sum()

In [ ]:
faturamento['cod_ap'] = faturamento['cod_ap'].astype(str)

In [ ]:
faturamento = pd.merge(faturamento, ibge, on='cod_ap', how='left')
faturamento.head(20)

In [ ]:
faturamento.isnull().sum()

In [ ]:
faturamento.dtypes

In [ ]:
faturamento.shape

In [ ]:
faturamento['cod_ap'].nunique()

In [ ]:
faturamento['cod_municipio'].nunique()

In [ ]:
faturamento['renda_media'] = faturamento['renda_media'].astype(str).str.translate({32:None, 44:46}).astype(float)
faturamento['ano'] = faturamento['ano'].astype(str)
faturamento['mes'] = faturamento['mes'].astype(str)
faturamento.dtypes

In [ ]:
# estado

cod_estado = faturamento['cod_municipio'].astype('str').str[:2]
cod_estado = cod_estado.replace({'11' : 'RO', '12' : 'AC', '13' : 'AM', '14' : 'RR', '15' : 'PA',
                                 '16' : 'AP', '17' : 'TO', '21' : 'MA', '22' : 'PI', '23' : 'CE',
                                 '24' : 'RN', '25' : 'PB', '26' : 'PE', '27' : 'AL', '28' : 'SE',
                                 '29' : 'BA', '31' : 'MG', '32' : 'ES', '33' : 'RJ', '35' : 'SP',
                                 '41' : 'PR', '42' : 'SC', '43' : 'RS', '50' : 'MS', '51' : 'MT',
                                 '52' : 'GO', '53' : 'DF'})
faturamento['estado'] =  cod_estado

# regiao 

cod_regiao = faturamento['estado'].astype('str').str[:2]
cod_regiao = cod_regiao.replace({'RO' : 'NT', 'AC' : 'NT', 'AM' : 'NT', 'RR' : 'NT', 'PA' : 'NT',
                                 'AP' : 'NT', 'TO' : 'NT', 'MA' : 'NE', 'PI' : 'NE', 'CE' : 'NE',
                                 'RN' : 'NE', 'PB' : 'NE', 'PE' : 'NE', 'AL' : 'NE', 'SE' : 'NE',
                                 'BA' : 'NE', 'MG' : 'SD', 'ES' : 'SD', 'RJ' : 'SD', 'PR' : 'SU',
                                 'SC' : 'SU', 'RS' : 'SU', 'MS' : 'CO', 'MT' : 'CO', 'MS' : 'CO',
                                 'GO' : 'CO', 'DF' : 'CO'})
faturamento['regiao'] =  cod_regiao


In [ ]:
faturamento.shape[0]

In [ ]:
faturamento.head(15)

In [ ]:
localidades = pd.read_csv('localidades.csv', sep=';', encoding='latin1')

In [ ]:
localidades.head(10)

In [ ]:
localidades.dtypes

In [ ]:
localidades = localidades.rename(columns={'CD_GEOCODSD': 'cod_municipio', 'LONG': 'long', 'LAT': 'lat' })
localidades = localidades.loc[:, ['cod_municipio', 'long', 'lat']]
# cod muicipio vem com trailing zeros
localidades['cod_municipio'] = localidades['cod_municipio'].astype(str).str[:7]
localidades['lat'] = localidades['lat'].astype(str).str.translate({32:None, 44:46}).astype(float)
localidades['long'] = localidades['long'].astype(str).str.translate({32:None, 44:46}).astype(float)

In [ ]:
localidades.head()

In [ ]:
# pegando a media de latitude e longitude das areas como aproximacao do muncipiio
localidades = localidades.groupby('cod_municipio').mean()
localidades.head()

In [ ]:
# latitude e longitude por municipio
faturamento = pd.merge(faturamento, localidades, on='cod_municipio', how='left')
faturamento.head()

In [ ]:
percent_missing = faturamento.isnull().sum() * 100 / len(faturamento)
missing_value_df = pd.DataFrame({'column_name': faturamento.columns,
                                 'percent_missing': percent_missing}).reset_index(drop=True)
missing_value_df

In [ ]:
# observacao importante : tem -1  em cod_ap, cod_municipio, regiao e estado. Por isso que aparece como se nao tivesse nenhum 
# missing value mas na vdd tem. Para substutui os missing values, samplear de uma gaussian fitada nos dados existentes uma boa?

# porem o fato de nao termos os dados diz algo a mais sobre essas lojas? tem relacao com a receita?
faturamento[faturamento['cod_ap'] == '-1'].sample(20)

In [ ]:
# EDA 
    # target variable and aproximate target variable (que a gente nao vai ter para submissao) 
    # pair anlysis
    # corr matrices
    # multicolinearity
    # etc

In [ ]:
    # Obsevacoes:
        
        # Check Correlations and Multicolinearity in data
             # no  test set só teremos cod_loja, mes_cont, cod_ap, cod_municipio, renda_media, lat, long, num_lojas_regiao
             # sabemos que temos colinearidade forte entre cod_ap e renda_media, e de cod_municipio com lat e long 
             # tambem poderemos ter colinearidade forte de cod_loja com as features da loja
             # ferramentas: - teste VIF de multicolinearidade; chi-square test for independece, anova, pearson correlation

In [ ]:
faturamento['cod_loja'].nunique()

In [ ]:
print(faturamento.info())
print(faturamento.dtypes)

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! parei aqui
# tem 167 lojas que tem meses faltando no ano
check_months = faturamento.groupby('cod_loja')['mes_cont'].nunique()
# tem que dar zero
print(len(check_months[check_months!=24]))

In [ ]:
faturamento['cod_loja'] = faturamento['cod_loja'].astype(int)
faturamento['ano'] = faturamento['ano'].astype(int)
faturamento['mes'] = faturamento['mes'].astype(int)
faturamento = faturamento.sort_values(by=['cod_loja', 'ano', 'mes'])
faturamento['cod_loja'] = faturamento['cod_loja'].astype(str)
faturamento['ano'] = faturamento['ano'].astype(str)
faturamento['mes'] = faturamento['mes'].astype(str)

In [ ]:
faturamento[216:240]

In [ ]:
# datasets por loja 
# !!! tem lojas com meses faltando !!!

faturamentos_lojas = [ faturamento[24*i:24*i+24] for i in range(3131)]

print(faturamentos_lojas[9]['mes'])

#pegando lojas aleatorias
for i in range(5):
    rand = np.random.randint(0, 3131)
    #plot something
    fig, ax = plt.subplots()
    dataframe_loja = faturamentos_lojas[rand]
    #print(dataframe_loja)
    dataframe_loja['receita'][:12].reset_index(drop=True).plot(kind='line', ax=ax)
    dataframe_loja['receita'][12:].reset_index(drop=True).plot(kind='line', ax=ax)
    plt.show()


In [ ]:
# analise global , media da receita das lojas ao longo do tempo

fig, ax = plt.subplots()
faturamanto_receita_media = faturamento.groupby('mes_cont')['receita'].mean().plot(kind='line', ax=ax)
plt.show()

# diferenca media entre os anos
faturamanto_receita_media_2016 = faturamanto_receita_media[:13]
faturamanto_receita_media_2017 = faturamanto_receita_media[13:]

In [ ]:
print('Lojas com geo desconhecido (-1) - média geral: {}'.format(faturamento[faturamento['cod_ap'] == '-1']['receita'].mean()))
print('Lojas com geo desconhecido - std geral: {}'.format(faturamento[faturamento['cod_ap'] == '-1']['receita'].std()))
print('----------------')
print('Lojas com geo conhecido - média geral: {}'.format(faturamento[faturamento['cod_ap'] != '-1']['receita'].mean()))
print('Lojas com geo conhecido - std geral: {}'.format(faturamento[faturamento['cod_ap'] != '-1']['receita'].std()))

# no conjunto de lojas -1 a media eh menor(mas parece bem pouco) e a variancia eh maior (talvez seja reelvante aqui)

# plotar as distribuicoes das receitas
receitas_geo_desconhecido = faturamento[faturamento['cod_ap'] == '-1']['receita']                                      
receitas_geo_conhecido = faturamento[faturamento['cod_ap'] != '-1']['receita']



In [ ]:
# rows which features from 13 to 18 are NaN

print('Lojas com NaN - média geral: {}'.format(faturamento[faturamento['feature_14'] != faturamento['feature_14']]['receita'].mean()))
print('Lojas com NaN - std geral: {}'.format(faturamento[faturamento['feature_14'] != faturamento['feature_14']]['receita'].std()))
print('----------------')
print('Lojas com features -  média geral:{}'.format(faturamento[faturamento['feature_14'] == faturamento['feature_14']]['receita'].mean()))
print('Lojas com features -  std geral:{}'.format(faturamento[faturamento['feature_14'] == faturamento['feature_14']]['receita'].std()))

#IMPORTANTE: Lojas que NaN nas features 13-18 tem receita mt menor!
# lojas quiosque talvez ?

# plotar as dsitribuicoes das receitas
# ver se as features estao influenciado a receita em faturamento_features_conhecidas (plotar)

receitas_features_NaN = faturamento[faturamento['feature_14'] != faturamento['feature_14']]['receita']
receitas_features_conhecidas = faturamento[faturamento['feature_14'] == faturamento['feature_14']]['receita']

In [ ]:
# influencia do numero de lojas na regiao
receita_regiao = faturamento.groupby('cod_ap').aggregate({'cod_loja': 'count','receita': 'mean'})
receita_regiao

In [ ]:
faturamento.to_csv('../processed/estrabico.csv')


In [ ]:
# target-value - receita
# Tem duas temporadas mesmo de receita? uma no meio do ano e outra no final? 
    #-> daria pra colocar feature binaria pros meses do meio do ano
# o ano eh importante?
# testar sazonalidade de trimestre 
# sazonalidade da receita pode ser diferente para cada loja/regiao/categoria?
#a avliar isso para a media de todas as lojas, media de todas as lojas por regiao
# 

In [ ]:
# lojas grandes e lojas normais? Existem grupos de lojas?

In [ ]:
# ver o comportamento de tipos de categorias presentes em cada mês 

In [ ]:
# como as categorias diferem em vendas e receita, e preço consequentemente?

In [ ]:
# como preço da categorias varia durante o tempo?  Inflação importante? 

In [ ]:
# agr sobre as features das lojas

# features 7 e 8 bem parecidas?
# feature 9 parece ter dois grupos definidos, um de valores maiores e outro de valores menores , ver isso aí
# se features 13-18 sao influentes (pq sempre que elas aparecem, aparecem juntas), e um binario nao eh suficiente:
    # feature 15 parece ter valores discretos bem definidos, relacionada a fração de alguma coisa, tipo x/10 * 1000 -> substituir por números de 1 a 10? Dividir em grupos maiores?
    # fature 16 parece ser binária, 500 ou 1000, tratar como binária? ou 1 e 2 ?
    # features 13, 15, 16 tem os valores máximos = moda = 1000 , colocar features binárias para a presença de max?
    # feature 4 parece ter 2 grupos de valores: altos e normais. Talvez a questao de lojas grande e normais?

In [ ]:
# automatic feature generayion pras features 1-12 (1 - 18 se as features  13-18 forem necessarias)

In [ ]:
# clustering (K-means with PCA / t-SNE)

In [ ]:
# EDA time series stuff (target variable time-series anlysis)
# Scipy 2019 tutorial



In [ ]:
# Subsitute Missing Values

In [ ]:
# Treat Outliers

In [ ]:
# feature selection and engineering   
    # - sabemos de antemao a "cara" dos resultados no comeco de ano pq a easonalidade é mt forte e a variancia eh pequena para meses
    # - por isso seria bom dar uma forma dele copiar um pouco o ano passo, serve como uma proxy
    # - feature categorica que vale: 'Janeiro'; 'Fevereiro', 'Marco', 'Outros'
    
    # feature binaria que vale 1 se as features 13 a 18 estao presentes e 0 caso contrario
    
    # feature trimestral, comeco do ano, meio, final
    
    # feature de numero de lojas na mesmo cod_ap 
    
    # rotacionar latitude e longitude -> ajuda o modelo de arvore

In [ ]:
# Encode Categorical Values
 # mean encodings (pure or rounded)

In [ ]:
# build lag_dataset (with lag variables/sliding window)

In [ ]:
# build receita_diff (differentiated time series of target value)
# plot it

In [ ]:
# lembrandoque vamos fazer a previsao do mes seguinte só, depois repetimos o mesmo processo para os proximos meses
# train-test split 
# test set vai ser os ultimos 3 meses

In [ ]:
# local cross validation function
     # lembrar de fazer sort by loja, pra fazer um stack das predictions [loja1: outubro2017 loja2: outubro2017 ...]
     # pq na hora de fazer as previsoes vamos empacotar desse jeito

In [ ]:
# global cross validation function (next three months)
    # prediction
    # for i in range(1,3):
    # faturamento_i+1 = append do train_i com prediction_i
    # fazer tudo denovo
        # train test split deixando ultimos 3 meses de fora
        # treinar e cv modelos, treinar e cv ensemble, treinar modelos no dataset inteiro, treinar e cv ensemble no dataset inteiro
        # conseguir prediction_i 
        # append prediction_i no prediction_months
    # if cv=True 
        #call local cross validation and return cv 
    # else 
        #return prediction_months

In [ ]:
# baseline model 1

# 0. Separar dataset para cada loja
# 1. Fazer 2017 - 2016
# 2. plotar 
# 3. fit and predict with ARIMA (diff = 1)
# 5. somar valor obtido com 2016
# 6. juntar as predictions
# 4. cv ARIMA

In [ ]:
# baseline model 2

# 0. Separar dataset para cada loja
# 1. fit and predict with ARIMA (diff = 1 )
# 2. somar com valor do mes passado
# 3  juntar predtions
# 4. cv ARIMA

In [ ]:
# xgboost with lag dataset

In [ ]:
# train xgboost with lag dataset parameter tuning

In [ ]:
# cv xgboost

In [ ]:
# train other models
 # remember that with linear models we have to normalize variables

In [ ]:
# other models parameter tuning (if needed)

In [ ]:
# cv other models

In [ ]:
# ensemble

In [ ]:
# cv ensemble

In [ ]:
# get parameters of ensemble, train models again on whole dataset

In [ ]:
# make predictions wth global cv function (cv=False) for submission